In [231]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.storagelevel import StorageLevel

In [205]:
spark=SparkSession.builder.appName('IVTasks').getOrCreate()

In [206]:
file_path="C:/SparkCourse/tensorIOT/Books_5.json"
json_df = spark.read.json(file_path)

In [198]:
json_df.count()

27164983

In [214]:
json_df=json_df.select(json_df['*'],col('style.*'))
json_df=json_df.drop("style")

In [217]:
for i in json_df.columns:
    if ':' in i:
        j=i.replace(" ","_").replace(":","")
        json_df=json_df.withColumnRenamed(i,j)

In [199]:
#transform date to MM-DD-YYYY format
json_df=json_df.withColumnRenamed("asin","item_id")
json_df=json_df.withColumn("formatted_date",from_unixtime(col("unixReviewTime"),"MM-dd-yyyy"))

In [200]:
#asin is item id
#grouping to find average of ratings
avg_df=json_df.groupBy("item_id").avg("overall")
avg_df=avg_df.withColumnRenamed("avg(overall)","avg_rating")

In [201]:
#item with least rating
avg_df.sort("avg_rating",ascending=True).show(1)

+----------+----------+
|   item_id|avg_rating|
+----------+----------+
|0470633735|       1.0|
+----------+----------+
only showing top 1 row



In [202]:
#item with most rating
avg_df.sort("avg_rating",ascending=False).show(1)

+----------+----------+
|   item_id|avg_rating|
+----------+----------+
|0007159862|       5.0|
+----------+----------+
only showing top 1 row



In [184]:
json_df=json_df.withColumn("review_len",size(split(json_df.reviewText,' ')))
#maximum review length
max_rev_len=json_df.select(max("review_len")).collect()[0][0]

In [185]:
#items having longest review
json_df.filter(json_df.review_len==max_rev_len).select("item_id").show()

+----------+
|   item_id|
+----------+
|B001IKJOLW|
|B0058YEJ5K|
|B0014F7B98|
|B009MA34NY|
|B0092UF54A|
|B005AGO4LU|
|B010RRWKT4|
|B014IBJKNO|
+----------+



In [189]:
#writing data to parquet format
json_df.write.mode("append").parquet("C:/SparkCourse/tensorIOT/")

Py4JJavaError: An error occurred while calling o599.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:110)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103

In [233]:
#desired data frame operation which I learnt recently.
#There multiple persist options available so choosing the MEMORY_AND_DISK will spill the data that cannot be handled in memory into DISK.


count_df=json_df.groupBy("reviewerName","Format",).count()
count_df.persist(StorageLevel.MEMORY_AND_DISK)
count_df.show()

+--------------------+---------------+-----+
|        reviewerName|         Format|count|
+--------------------+---------------+-----+
|   Sandra L. Wooster|      Hardcover|    6|
|        B. Valentine|      Hardcover|   46|
|               Lee K|      Paperback|   34|
|            scarlett| Kindle Edition|   62|
|            willie b|      Hardcover|    1|
|            A. Vegan|      Paperback|  115|
|        Erik Fransen| Kindle Edition|    6|
|               Jerry|      Hardcover|  299|
|          Cat Smiles|      Paperback|   33|
|  John H. Kuhl, CPCM| Kindle Edition|  154|
|            R. Albin|      Hardcover|  542|
|   Mary L. Jonilonis| Kindle Edition|   15|
|        Caleb Kruger| Kindle Edition|   35|
|      Robert Zaleski|      Paperback|   21|
|Mr. Nathaniel Singer|      Paperback|    6|
|             Sindris| Kindle Edition|   11|
|         josh morgan|      Paperback|    5|
|     Susan K Hillmer|      Hardcover|    5|
|                  SV| Kindle Edition|  257|
|       Ph

In [194]:
jdbc_url="jdbc:sqlite:C://ProgramFiles//de2020//sql"
#writing df to 
json_df.write.format('jdbc').options(
  url=jdbc_url,
  driver='org.postgresql.Driver',
  dbtable='table',
  user='xxxx',
  password='xxxx').mode('append').save()